In [1]:
## Testing multiple neural net architectures on features

In [1]:
#Import necessary libraries

import pandas as pd
import numpy as np
from scipy import signal
import os
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pickle as pkl
import itertools 
import glob
from sklearn import svm 
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix, f1_score
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

%matplotlib inline 
# %matplotlib qt


In [2]:
# Import AP dataset
AP_df = pd.read_pickle("F:\EEG-data\\think-count\\featureExtraction\AUC_PSD\AUC_PSD_df_01234.pkl")
print(AP_df.shape)

(640, 801)


In [ ]:
# Import AP dataset, theta and alpha bands only 
taAP_df = pd.read_pickle("F:\EEG-data\\think-count\\featureExtraction\AUC_PSD\AUC_PSD_df_12.pkl")
print(taAP_df.shape)

In [3]:
X = AP_df.iloc[:,:-1].values.astype(np.float32)
class_names = ["T","C"]
y = AP_df.iloc[:,-1].apply(class_names.index).values.astype(np.float32)

print(X.shape)
print(y.shape)

(640, 800)
(640,)


In [ ]:
#Theta and alpha bands only
taX = taAP_df.iloc[:,:-1].values.astype(np.float32)
class_names = ["T","C"]
tay = taAP_df.iloc[:,-1].apply(class_names.index).values.astype(np.float32)

print(taX.shape)
print(tay.shape)

In [4]:
#Split into train and test sets
def split_train_test(X,y):
    sss = StratifiedShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
    for train_index, test_index in sss.split(X,y):
                x_train, x_test = X[train_index],X[test_index]
                y_train, y_test = y[train_index],y[test_index]
    return x_train,x_test,y_train,y_test

In [5]:
#0.1s
x_train01,x_test01,y_train01,y_test01 = split_train_test(X,y)

x_train01 = x_train01.reshape(512,800,1)
x_test01 = x_test01.reshape(128,800,1)

print(x_train01.shape)
print(x_test01.shape)
print(y_train01.shape)
print(y_test01.shape)

(512, 800, 1)
(128, 800, 1)
(512,)
(128,)


In [ ]:
#0.1s theta and alpha bands
tax_train01,tax_test01,tay_train01,tay_test01 = split_train_test(taX,tay)

tax_train01 = tax_train01.reshape(512,320,1)
tax_test01 = tax_test01.reshape(128,320,1)

print(tax_train01.shape)
print(tax_test01.shape)
print(tay_train01.shape)
print(tay_test01.shape)

In [6]:
#DNN model
def DNN_model(no_features):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(no_features, input_shape = (no_features,1), activation="relu"),
        tf.keras.layers.Dense(10, activation="relu"), 
        tf.keras.layers.Dense(1,activation="sigmoid")
    ])
    model.compile(loss="binary_crossentropy",
                optimizer='adam',
                metrics=["accuracy"])
    # model.summary()
    return model

#Conv model
def CONV_model(no_features):
    model2 = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=128, kernel_size=5,strides=1, padding="causal",
                                activation="relu",input_shape=(no_features,1)), #input shape = (n_timesteps,n_features)
        tf.keras.layers.Conv1D(filters=64, kernel_size=5,strides=1, padding="causal",activation="relu"),
        tf.keras.layers.Conv1D(filters=32, kernel_size=5,strides=1, padding="causal",activation="relu"),
        # tf.keras.layers.Dropout(0.5),
        tf.keras.layers.MaxPool1D(pool_size=2),
        tf.keras.layers.Conv1D(filters=16, kernel_size=5,strides=1, padding="causal",activation="relu"),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation="relu"), 
        # tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1,activation="sigmoid")
    ])
    # optimizer = tf.keras.optimizers.Adam(lr=9e-4)
    model2.compile(loss="binary_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])
    return model2

#Conv + LSTM
def CONV_LSTM_model(no_features):
    model3 = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=128, kernel_size=5,strides=1, padding="causal",
                              activation="relu",input_shape=(no_features,1)), #input shape = (n_timesteps,n_features)
    tf.keras.layers.Conv1D(filters=64, kernel_size=5,strides=1, padding="causal",activation="relu"),
    tf.keras.layers.Conv1D(filters=32, kernel_size=5,strides=1, padding="causal",activation="relu"),
    # tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPool1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=16, kernel_size=5,strides=1, padding="causal",activation="relu"),
    # tf.keras.layers.Flatten(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Flatten(),
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Dense(10, activation="relu"), 
    # tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1,activation="sigmoid")
])
    model3.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=["accuracy"])
    return model3

#Simple RNN 
def RNN_model(no_features):
    model4 = tf.keras.models.Sequential([
    tf.keras.layers.SimpleRNN(40, input_shape = (no_features,1),return_sequences=True),
    tf.keras.layers.SimpleRNN(40),
    # tf.keras.layers.Dense(10,activation="relu",input_shape=(12,32)),
    tf.keras.layers.Dense(1,activation="sigmoid")
    ])
    model4.compile(loss="binary_crossentropy",
                optimizer='adam',
                metrics=["accuracy"])
    return model4

#LSTM only

def LSTM_model(no_features):
    model5 = tf.keras.models.Sequential([
    # tf.keras.layers.Flatten(input_shape=(sampLen,32)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True,input_shape=(no_features,1))),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10,activation="relu"),
    tf.keras.layers.Dense(1,activation="sigmoid")
    ])
    model5.compile(loss="binary_crossentropy",
                optimizer='adam',
                metrics=["accuracy"])
    return model5

In [7]:
def test(model,dataset,devset,batchsize,epochs):
    tf.random.set_seed(10)
    datasetb = dataset.batch(batchsize)
    devsetb = devset.batch(batchsize)
    history = model.fit(datasetb,epochs=epochs,verbose=0,validation_data=devsetb)
    return history 

In [8]:
# 0.1s 
dataset01 = tf.data.Dataset.from_tensor_slices((x_train01,y_train01))
devset01  = tf.data.Dataset.from_tensor_slices((x_test01,y_test01))

In [ ]:
# 0.1s theta and alpha bands only
dataset01_ta = tf.data.Dataset.from_tensor_slices((tax_train01,tay_train01))
devset01_ta  = tf.data.Dataset.from_tensor_slices((tax_test01,tay_test01))

In [10]:
# 0.1s 
# Testing different models, epochs=10, batchsize = 40
batchsize = 1
no_features = 800
DNN_model01 = DNN_model(no_features)
CONV_model01 = CONV_model(no_features)
CONV_LSTM_model01 = CONV_LSTM_model(no_features)
LSTM_model01 = LSTM_model(no_features)
RNN_model01 = RNN_model(no_features)
#Histories 
DNN_hist01 = test(DNN_model01,dataset01,devset01,batchsize,10)
CONV_hist01 = test(CONV_model01,dataset01,devset01,batchsize,10)
CONV_LSTM_hist01 = test(CONV_LSTM_model01,dataset01,devset01,batchsize,10)
LSTM_hist01 = test(LSTM_model01,dataset01,devset01,batchsize,10)
RNN_hist01 = test(RNN_model01,dataset01,devset01,batchsize,10)
#Max validation accuracy 
valacc_DNN01 = max(DNN_hist01.history['val_accuracy'])
valacc_CONV01 = max(CONV_hist01.history['val_accuracy'])
valacc_CONVLSTM01 = max(CONV_LSTM_hist01.history['val_accuracy'])
valacc_LSTM01 = max(LSTM_hist01.history['val_accuracy'])
valacc_RNN01 = max(RNN_hist01.history['val_accuracy'])

print(valacc_DNN01)
print(valacc_CONV01)
print(valacc_CONVLSTM01)
print(valacc_LSTM01)
print(valacc_RNN01)

0.5
0.7109375
0.5
0.703125
0.640625


In [ ]:
# 0.1s theta and alpha bands only
# 0.1s 
# Testing different models, epochs=10, batchsize = 40
batchsize = 1
no_features = 320
DNN_model01 = DNN_model(no_features)
CONV_model01 = CONV_model(no_features)
CONV_LSTM_model01 = CONV_LSTM_model(no_features)
LSTM_model01 = LSTM_model(no_features)
RNN_model01 = RNN_model(no_features)
#Histories 
DNN_hist01 = test(DNN_model01,dataset01_ta,devset01_ta,batchsize,10)
CONV_hist01 = test(CONV_model01,dataset01_ta,devset01_ta,batchsize,10)
CONV_LSTM_hist01 = test(CONV_LSTM_model01,dataset01_ta,devset01_ta,batchsize,10)
LSTM_hist01 = test(LSTM_model01,dataset01_ta,devset01_ta,batchsize,10)
RNN_hist01 = test(RNN_model01,dataset01_ta,devset01_ta,batchsize,10)
#Max validation accuracy 
valacc_DNN01 = max(DNN_hist01.history['val_accuracy'])
valacc_CONV01 = max(CONV_hist01.history['val_accuracy'])
valacc_CONVLSTM01 = max(CONV_LSTM_hist01.history['val_accuracy'])
valacc_LSTM01 = max(LSTM_hist01.history['val_accuracy'])
valacc_RNN01 = max(RNN_hist01.history['val_accuracy'])

print(valacc_DNN01)
print(valacc_CONV01)
print(valacc_CONVLSTM01)
print(valacc_LSTM01)
print(valacc_RNN01)